# **Chapter 2: Our First Model**

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from PIL import Image

In [2]:
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False

Set up the transforms for every image:

Resize to 64x64
Convert to tensor
Normalize using ImageNet mean & std

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
img_transforms = transforms.Compose([
    transforms.Resize((64,64)),    
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225] )
    ])

In [5]:
train_data_path = "/content/drive/MyDrive/cats_vs_dogs_small/train/"
train_data = torchvision.datasets.ImageFolder(root=train_data_path,transform=img_transforms, is_valid_file=check_image)

In [6]:
val_data_path = "/content/drive/MyDrive/cats_vs_dogs_small/validation/"
val_data = torchvision.datasets.ImageFolder(root=val_data_path,transform=img_transforms, is_valid_file=check_image)

In [7]:
test_data_path = "/content/drive/MyDrive/cats_vs_dogs_small/test/"
test_data = torchvision.datasets.ImageFolder(root=test_data_path,transform=img_transforms, is_valid_file=check_image)

In [8]:
batch_size=64

In [9]:
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
val_data_loader  = torch.utils.data.DataLoader(val_data, batch_size=batch_size) 
test_data_loader  = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

In [18]:
list(train_data_loader)

[[tensor([[[[ 1.4269,  1.5297,  1.6667,  ...,  2.1290,  2.1119,  2.0434],
            [ 1.4269,  1.5297,  1.6667,  ...,  2.1290,  2.1119,  2.0605],
            [ 1.4440,  1.5468,  1.6667,  ...,  2.1119,  2.1119,  2.0948],
            ...,
            [ 0.6734,  0.7248,  0.7933,  ..., -2.0665, -2.0665, -2.0665],
            [ 0.6221,  0.6734,  0.7419,  ..., -2.0665, -2.0665, -2.0665],
            [ 0.5364,  0.6049,  0.6906,  ..., -2.0837, -2.0837, -2.0837]],
  
           [[ 0.9055,  1.0105,  1.0980,  ...,  1.7283,  1.6232,  1.5357],
            [ 0.9055,  0.9930,  1.0980,  ...,  1.7633,  1.6758,  1.5882],
            [ 0.8880,  0.9755,  1.0805,  ...,  1.7808,  1.7283,  1.6583],
            ...,
            [ 0.2227,  0.2752,  0.3102,  ..., -1.9657, -1.9657, -1.9832],
            [ 0.1702,  0.2227,  0.2577,  ..., -1.9657, -1.9657, -1.9832],
            [ 0.1352,  0.1877,  0.2402,  ..., -2.0007, -2.0007, -2.0007]],
  
           [[-0.2184, -0.1312, -0.0441,  ...,  0.6008,  0.4439,  0.339

# Our First Model, SimpleNet
SimpleNet is a very simple combination of three Linear layers and ReLu activations between them. Note that as we don't do a softmax() in our forward(), we will need to make sure we do it in our training function during the validation phase.

In [10]:
class SimpleNet(nn.Module):

    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(12288, 84)
        self.fc2 = nn.Linear(84, 50)
        self.fc3 = nn.Linear(50,2)
    
    def forward(self, x):
        x = x.view(-1, 12288)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [11]:
simplenet = SimpleNet()

# Create an optimizer
Here, we're just using Adam as our optimizer with a learning rate of 0.001.

In [12]:
optimizer = optim.Adam(simplenet.parameters(), lr=0.001)

# **Copy the model to GPU**
Copy the model to the GPU if available.

In [13]:
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

simplenet.to(device)

SimpleNet(
  (fc1): Linear(in_features=12288, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)

# Training
Trains the model, copying batches to the GPU if required, calculating losses, optimizing the network and perform validation for each epoch.

In [20]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="gpu"):
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output), dim=1)[1], targets).view(-1)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

In [21]:
train(simplenet, optimizer,torch.nn.CrossEntropyLoss(), train_data_loader,val_data_loader, epochs=10, device=device)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 0, Training Loss: 0.68, Validation Loss: 0.69, accuracy = 0.51
Epoch: 1, Training Loss: 0.67, Validation Loss: 0.68, accuracy = 0.52
Epoch: 2, Training Loss: 0.67, Validation Loss: 0.69, accuracy = 0.52
Epoch: 3, Training Loss: 0.66, Validation Loss: 0.69, accuracy = 0.52
Epoch: 4, Training Loss: 0.66, Validation Loss: 0.70, accuracy = 0.52
Epoch: 5, Training Loss: 0.65, Validation Loss: 0.69, accuracy = 0.52
Epoch: 6, Training Loss: 0.65, Validation Loss: 0.69, accuracy = 0.52
Epoch: 7, Training Loss: 0.64, Validation Loss: 0.69, accuracy = 0.51
Epoch: 8, Training Loss: 0.64, Validation Loss: 0.69, accuracy = 0.51
Epoch: 9, Training Loss: 0.62, Validation Loss: 0.82, accuracy = 0.51


# Making predictions

In [24]:
labels = ['cat','dog']

img = Image.open("/content/drive/MyDrive/cats_vs_dogs_small/test/cats/cat.3003.jpg") 
img = img_transforms(img).to(device)


prediction = F.softmax(simplenet(img))
prediction = prediction.argmax()
print(labels[prediction])

dog


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  import sys


# Saving Models
We can either save the entire model using save or just the parameters using state_dict. Using the latter is normally preferable, as it allows you to reuse parameters even if the model's structure changes (or apply parameters from one model to another).

In [ ]:
torch.save(simplenet, "/tmp/simplenet") 
simplenet = torch.load("/tmp/simplenet")

In [ ]:
torch.save(simplenet.state_dict(), "/tmp/simplenet")    
simplenet = SimpleNet()
simplenet_state_dict = torch.load("/tmp/simplenet")
simplenet.load_state_dict(simplenet_state_dict)